### step0) Colab 환경 설정

In [ ]:
# https://colab.research.google.com/github/showlab/Tune-A-Video/blob/main/notebooks/Tune-A-Video.ipynb#scrollTo=jjcSXTp-u-Eg

In [ ]:
#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

NVIDIA A100-SXM4-40GB, 40960 MiB, 40513 MiB


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Install requirements

!git clone https://github.com/showlab/Tune-A-Video.git /content/drive/MyDrive/TAV-KO
%cd /content/drive/MyDrive/TAV-KO
# %pip install -r requirements.txt
%pip install -q -U --pre triton
%pip install -q diffusers[torch]==0.11.1 transformers==4.26.0 bitsandbytes==0.35.4 \
decord accelerate omegaconf einops ftfy gradio imageio-ffmpeg xformers

In [ ]:
%cd ./

In [ ]:
#@title Download pretrained model

#@markdown Name/Path of the initial model.
MODEL_NAME = "Bingsu/my-korean-stable-diffusion-v1-5" #@param {type:"string"}

#@markdown If model should be download from a remote repo. Untick it if the model is loaded from a local path.
download_pretrained_model = True #@param {type:"boolean"}
if download_pretrained_model:
    !git lfs install
    !git clone https://huggingface.co/$MODEL_NAME checkpoints/$MODEL_NAME
    MODEL_NAME = f"./checkpoints/{MODEL_NAME}"
print(f"[*] MODEL_NAME={MODEL_NAME}")

In [ ]:
#@markdown If model weights should be saved directly in google drive (takes around 4-5 GB).
save_to_gdrive = False #@param {type:"boolean"}
if save_to_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')

#@markdown Enter the directory name to save model at.

OUTPUT_DIR = "outputs/man-surfing" #@param {type:"string"}
if save_to_gdrive:
    OUTPUT_DIR = "/content/drive/MyDrive/" + OUTPUT_DIR

print(f"[*] Weights will be saved at {OUTPUT_DIR}")

!mkdir -p $OUTPUT_DIR

## Training

In [ ]:
#@markdown Train config

from omegaconf import OmegaConf

CONFIG_NAME = "configs/man-surfing.yaml" #@param {type:"string"}

train_video_path = "data/man-surfing.mp4" #@param {type:"string"}
train_prompt = "a man is surfing" #@param {type:"string"} # Must korean (or english)
video_length = 24 #@param {type:"number"}
width = 512 #@param {type:"number"}
height = 512 #@param {type:"number"}
learning_rate = 3e-5 #@param {type:"number"}
train_steps = 400 #@param {type:"number"} // paper 500 steps

config = {
  "pretrained_model_path": MODEL_NAME,
  "output_dir": OUTPUT_DIR,
  "train_data": {
    "video_path": train_video_path,
    "prompt": train_prompt,
    "n_sample_frames": video_length,
    "width": width,
    "height": height,
    "sample_start_idx": 0,
    "sample_frame_rate": 2,
  },
  "validation_data": {
    "prompts": [
      "호랑이가 수박을 먹고 있습니다", # you can change this sentences!
      "토끼가 오렌지를 먹고 있습니다",
      "토끼가 피자를 먹고 있습니다",
      "강아지가 오렌지를 먹고 있습니다",
    ],
    "video_length": video_length,
    "width": width,
    "height": height,
    "num_inference_steps": 20,
    "guidance_scale": 12.5,
    "use_inv_latent": True,
    "num_inv_steps": 50,
  },
  "learning_rate": learning_rate,
  "train_batch_size": 1,
  "max_train_steps": train_steps,
  "checkpointing_steps": 1000,
  "validation_steps": 100,
  "trainable_modules": [
    "attn1.to_q",
    "attn2.to_q",
    "attn_temp",
  ],
  "seed": 33,
  "mixed_precision": "fp16",
  "use_8bit_adam": False,
  "gradient_checkpointing": True,
  "enable_xformers_memory_efficient_attention": True,
}

OmegaConf.save(config, CONFIG_NAME)

In [ ]:
# If TypeError: The keyword `fps` is no longer supported. Use `duration`(in ms) instead, e.g. `fps=50` == `duration=20` (1000 * 1/50),
'''
In, /content/drive/MyDrive/TAV-KO/tuneavideo/util.py
imageio.mimsave(path, outputs, fps=fps)
--> imageio.mimsave(path, outputs, duration=fps)
'''

In [ ]:
!accelerate launch train_tuneavideo.py --config=$CONFIG_NAME

## Inference

In [ ]:
import torch
from torch import autocast
from diffusers import DDIMScheduler
from tuneavideo.pipelines.pipeline_tuneavideo import TuneAVideoPipeline
from tuneavideo.models.unet import UNet3DConditionModel
from tuneavideo.util import save_videos_grid


unet = UNet3DConditionModel.from_pretrained(OUTPUT_DIR, subfolder='unet', torch_dtype=torch.float16).to('cuda')
scheduler = DDIMScheduler.from_pretrained(MODEL_NAME, subfolder='scheduler')
pipe = TuneAVideoPipeline.from_pretrained(MODEL_NAME, unet=unet, scheduler=scheduler, torch_dtype=torch.float16).to("cuda")
pipe.enable_xformers_memory_efficient_attention()
pipe.enable_vae_slicing()

g_cuda = None

In [ ]:
#@markdown Run for generating videos.

prompt = "흰색 옷을 입은 남자가 바다를 걷고 있습니다" #@param {type:"string"}
negative_prompt = "" #@param {type:"string"}
use_inv_latent = True #@param {type:"boolean"}
inv_latent_path = "" #@param {type:"string"}
num_samples = 1 #@param {type:"number"}
guidance_scale = 12.5 #@param {type:"number"}
num_inference_steps = 50 #@param {type:"number"}
video_length = 24 #@param {type:"number"}
height = 512 #@param {type:"number"}
width = 512 #@param {type:"number"}

ddim_inv_latent = None
if use_inv_latent and inv_latent_path == "":
    from natsort import natsorted
    from glob import glob
    import os
    inv_latent_path = natsorted(glob(f"{OUTPUT_DIR}/inv_latents/*"))[-1]
    ddim_inv_latent = torch.load(inv_latent_path).to(torch.float16)
    print(f"DDIM inversion latent loaded from {inv_latent_path}")

with autocast("cuda"), torch.inference_mode():
    videos = pipe(
        prompt,
        latents=ddim_inv_latent,
        video_length=video_length,
        height=height,
        width=width,
        negative_prompt=negative_prompt,
        num_videos_per_prompt=num_samples,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=g_cuda
    ).videos

save_dir = "./results" #@param {type:"string"}
save_path = f"{save_dir}/{prompt}.gif"
save_videos_grid(videos, save_path)

# display
from IPython.display import Image, display
display(Image(filename=save_path))

In [ ]:
#@markdown Free runtime memory
exit()